In [1]:
import pandas as pd

In [29]:
import glob
import os

In [ ]:
import pandas as pd
import glob
import os

directory_path_train = './mfcc_post_processing/'
directory_path_test = './mfcc_post_processing_test/'
file_pattern = "*.csv"
csv_files_train = glob.glob(os.path.join(directory_path_train, file_pattern))
csv_files_test = glob.glob(os.path.join(directory_path_test, file_pattern))
dataframes_train = []
dataframes_test = []

for file in csv_files_train:
    df = pd.read_csv(file)
    dataframes_train.append(df)
pd_train = pd.concat(dataframes_train, ignore_index=True)
print(pd_train.head()) 
print(pd_train.shape)
for file in csv_files_test:
    df = pd.read_csv(file)
    dataframes_test.append(df)
pd_test = pd.concat(dataframes_test, ignore_index=True)
print(pd_test.head())
print(pd_test.shape)

In [2]:
data_path = './mfcc_post_processing/1742_mfccs.csv'
df = pd.read_csv(data_path)

In [3]:
print(df.head())

   Coefficients          1          2          3          4          5  \
0    -558.47455   2.793246   2.786327   2.774820   2.758777   2.738251   
1    -555.69570   6.715124   6.683780   6.631803   6.559573   6.467629   
2    -553.84973   9.316228   9.256683   9.158134   9.021615   8.848545   
3    -552.81780  10.768115  10.686591  10.551867  10.365642  10.130268   
4    -552.09300  11.773888  11.635872  11.411560  11.109087  10.739025   

           6         7         8         9        10        11        12  \
0   2.713328  2.684097  2.650674  2.613182  2.571767  2.526580  2.477794   
1   6.356648  6.227449  6.080969  5.918281  5.740552  5.549061  5.345159   
2   8.640726  8.400292  8.129717  7.831718  7.509308  7.165675  6.804174   
3   9.848681  9.524376  9.161339  8.763979  8.337059  7.885606  7.414847   
4  10.313629  9.846010  9.349272  8.835732  8.316168  7.799348  7.291704   

   Instruments  
0            0  
1            0  
2            3  
3            3  
4            

In [4]:
features = df.drop('Instruments', axis=1)

In [5]:
labels = df['Instruments']

In [6]:
features.head()

,Coefficients,1,2,3,4,5,6,7,8,9,10,11,12
0,-558.47455,2.793246,2.786327,2.774820,2.758777,2.738251,2.713328,2.684097,2.650674,2.613182,2.571767,2.526580,2.477794
1,-555.69570,6.715124,6.683780,6.631803,6.559573,6.467629,6.356648,6.227449,6.080969,5.918281,5.740552,5.549061,5.345159
2,-553.84973,9.316228,9.256683,9.158134,9.021615,8.848545,8.640726,8.400292,8.129717,7.831718,7.509308,7.165675,6.804174
3,-552.81780,10.768115,10.686591,10.551867,10.365642,10.130268,9.848681,9.524376,9.161339,8.763979,8.337059,7.885606,7.414847
4,-552.09300,11.773888,11.635872,11.411560,11.109087,10.739025,10.313629,9.846010,9.349272,8.835732,8.316168,7.799348,7.291704


In [7]:
labels.head()

0    0
1    0
2    3
3    3
4    3
Name: Instruments, dtype: int64

In [8]:
from sklearn.preprocessing import OneHotEncoder

In [9]:
encoder = OneHotEncoder(sparse=False)
labels_encoded = encoder.fit_transform(labels.values.reshape(-1, 1))
print(labels_encoded.shape)

(79945, 2)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

In [11]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convert to PyTorch tensors
train_features = torch.tensor(X_train.values).float()
train_labels = torch.tensor(y_train).float()
val_features = torch.tensor(X_val.values).float()
val_labels = torch.tensor(y_val).float()

# Create TensorDatasets
train_dataset = TensorDataset(train_features, train_labels)
val_dataset = TensorDataset(val_features, val_labels)

# Create DataLoaders
batch_size = 32  # You can adjust the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [15]:
from torch import nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class InstrumentClassifier(nn.Module):
    def __init__(self, num_features, num_classes, dim_model=64, nhead=4, num_layers=2, dropout=0.1):
        super(InstrumentClassifier, self).__init__()
        self.embedding = nn.Linear(num_features, dim_model)
        encoder_layers = TransformerEncoderLayer(d_model=dim_model, nhead=nhead, dropout=dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, num_layers=num_layers)
        self.output_layer = nn.Linear(dim_model, num_classes)

    def forward(self, src):
        embedded = self.embedding(src)
        transformer_output = self.transformer_encoder(embedded)
        if transformer_output.dim() == 3:
            output = self.output_layer(transformer_output[:, 0, :])  # Correct if seq_length dimension exists
        else:
            output = self.output_layer(transformer_output)  # Use directly if no seq_length dimension
        return output

# Create a model instance
model = InstrumentClassifier(num_features=train_features.shape[1], num_classes=labels_encoded.shape[1])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [17]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Set device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assuming the model and dataloaders are already defined and instantiated
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training and validation phases
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# Number of epochs
num_epochs = 30

# Loop over the dataset multiple times
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss = validate_epoch(model, val_loader, criterion, device)
    
    print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

print('Finished Training')


Epoch 1/30 - Train Loss: 0.0622, Val Loss: 0.0505
Epoch 2/30 - Train Loss: 0.0526, Val Loss: 0.0504
Epoch 3/30 - Train Loss: 0.0519, Val Loss: 0.0469
Epoch 4/30 - Train Loss: 0.0523, Val Loss: 0.0560
Epoch 5/30 - Train Loss: 0.0505, Val Loss: 0.0480
Epoch 6/30 - Train Loss: 0.0490, Val Loss: 0.0484
Epoch 7/30 - Train Loss: 0.0487, Val Loss: 0.0480
Epoch 8/30 - Train Loss: 0.0476, Val Loss: 0.0492
Epoch 9/30 - Train Loss: 0.0546, Val Loss: 0.0486
Epoch 10/30 - Train Loss: 0.0502, Val Loss: 0.0468
Epoch 11/30 - Train Loss: 0.0480, Val Loss: 0.0574
Epoch 12/30 - Train Loss: 0.0472, Val Loss: 0.0467
Epoch 13/30 - Train Loss: 0.0466, Val Loss: 0.0486
Epoch 14/30 - Train Loss: 0.0460, Val Loss: 0.0538
Epoch 15/30 - Train Loss: 0.0459, Val Loss: 0.0573
Epoch 16/30 - Train Loss: 0.0463, Val Loss: 0.0444
Epoch 17/30 - Train Loss: 0.0429, Val Loss: 0.0441
Epoch 18/30 - Train Loss: 0.0416, Val Loss: 0.0396
Epoch 19/30 - Train Loss: 0.0422, Val Loss: 0.0382
Epoch 20/30 - Train Loss: 0.0398, Val Lo

In [27]:
data_path = './mfcc_post_processing/1749_mfccs.csv'
df = pd.read_csv(data_path)
features = df.drop('Instruments', axis=1)
labels = df['Instruments']

# Encode labels
encoder = OneHotEncoder(sparse=False)
labels_encoded = encoder.fit_transform(labels.values.reshape(-1, 1))

# Retrieve class names from the encoder
class_names = [str(name) for name in encoder.categories_[0]]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [28]:
from sklearn.metrics import accuracy_score, classification_report

test_features = torch.tensor(features.values).float()
test_labels = torch.tensor(labels_encoded).float()

# Create TensorDataset and DataLoader
test_dataset = TensorDataset(test_features, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def test_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            _, labels_indices = torch.max(labels, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels_indices.cpu().numpy())

    # Calculate accuracy and other classification metrics
    accuracy = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=class_names)
    return accuracy, report

# Execute the testing function
accuracy, report = test_model(model, test_loader, device)
print(f"Test Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)

Test Accuracy: 0.6809
Classification Report:
               precision    recall  f1-score   support

           0       0.33      0.18      0.23     16230
           1       0.74      0.87      0.80     43986

    accuracy                           0.68     60216
   macro avg       0.54      0.52      0.52     60216
weighted avg       0.63      0.68      0.65     60216

